<a href="https://colab.research.google.com/github/emmelinetsen/NLP/blob/master/NLP_NLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
#basic data management libraries
import pandas as pd
import numpy as np
import pylab as pl
import re
from urllib.request import urlopen


import nltk 
import string
import en_core_web_sm
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.text import Text
from textblob import TextBlob
from nltk.corpus import stopwords
from gensim import corpora


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

/usr/local/lib/python3.6/dist-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# Get to link to csv
url = 'https://drive.google.com/open?id=1tGR408tHc-3rgUX9TdAhtE2xqLviqMGO'

fluff, id = url.split('=')
print (id) # Verify that you have everything after '='

1tGR408tHc-3rgUX9TdAhtE2xqLviqMGO


In [5]:
# Store in a Pandas Dataframe
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('test.csv')  
df = pd.read_csv('test.csv')
df.head()

,ID,Label,Statement,Subject,Speaker,Title,State,Affiliation,barely-true count,false count,half-true count,mostly-true count,pants-fire count,Context
0,11972.json,TRUE,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview
1,11685.json,FALSE,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference
2,11096.json,FALSE,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show
4,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video


In [47]:
df = df[:10]
df

,ID,Label,Statement,Subject,Speaker,Title,State,Affiliation,barely-true count,false count,half-true count,mostly-true count,pants-fire count,Context
0,11972.json,TRUE,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview
1,11685.json,FALSE,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference
2,11096.json,FALSE,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show
4,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video
5,5962.json,TRUE,Over the past five years the federal governmen...,"federal-budget,pensions,retirement",brendan-doherty,NaN,Rhode Island,republican,1,2,1,1,0,a campaign website
6,7070.json,TRUE,Says that Tennessee law requires that schools ...,"county-budget,county-government,education,taxes",stand-children-tennessee,Child and education advocacy organization.,Tennessee,none,0,0,0,0,0,in a post on Facebook.
7,1046.json,barely-true,"Says Vice President Joe Biden ""admits that the...","economy,stimulus",john-boehner,Speaker of the House of Representatives,Ohio,republican,13,22,11,4,2,a press release.
8,12849.json,TRUE,Donald Trump is against marriage equality. He ...,"gays-and-lesbians,marriage",sean-patrick-maloney,Congressman for NY-18,New York,democrat,0,0,0,0,0,a speech at the Democratic National Convention
9,13270.json,barely-true,We know that more than half of Hillary Clinton...,foreign-policy,mike-pence,Governor,Indiana,republican,8,10,12,5,0,"comments on ""Meet the Press"""


In [48]:
text = df['Statement'][:10]
text

0    Building a wall on the U.S.-Mexico border will...
1    Wisconsin is on pace to double the number of l...
2    Says John McCain has done nothing to help the ...
3    Suzanne Bonamici supports a plan that will cut...
4    When asked by a reporter whether hes at the ce...
5    Over the past five years the federal governmen...
6    Says that Tennessee law requires that schools ...
7    Says Vice President Joe Biden "admits that the...
8    Donald Trump is against marriage equality. He ...
9    We know that more than half of Hillary Clinton...
Name: Statement, dtype: object

In [10]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [42]:
for w_index in text:
    print(wordpunct_tokenize(w_index))

['Building', 'a', 'wall', 'on', 'the', 'U', '.', 'S', '.-', 'Mexico', 'border', 'will', 'take', 'literally', 'years', '.']
['Wisconsin', 'is', 'on', 'pace', 'to', 'double', 'the', 'number', 'of', 'layoffs', 'this', 'year', '.']
['Says', 'John', 'McCain', 'has', 'done', 'nothing', 'to', 'help', 'the', 'vets', '.']
['Suzanne', 'Bonamici', 'supports', 'a', 'plan', 'that', 'will', 'cut', 'choice', 'for', 'Medicare', 'Advantage', 'seniors', '.']
['When', 'asked', 'by', 'a', 'reporter', 'whether', 'hes', 'at', 'the', 'center', 'of', 'a', 'criminal', 'scheme', 'to', 'violate', 'campaign', 'laws', ',', 'Gov', '.', 'Scott', 'Walker', 'nodded', 'yes', '.']
['Over', 'the', 'past', 'five', 'years', 'the', 'federal', 'government', 'has', 'paid', 'out', '$', '601', 'million', 'in', 'retirement', 'and', 'disability', 'benefits', 'to', 'deceased', 'former', 'federal', 'employees', '.']
['Says', 'that', 'Tennessee', 'law', 'requires', 'that', 'schools', 'receive', 'half', 'of', 'proceeds', '--', '$', '

In [50]:
# Creating new column to tokenize text
df['tokenized_statement'] = df.apply(lambda row: nltk.word_tokenize(row['Statement']), axis=1)
df

,ID,Label,Statement,Subject,Speaker,Title,State,Affiliation,barely-true count,false count,half-true count,mostly-true count,pants-fire count,Context,tokenized_statement
0,11972.json,TRUE,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview,"[Building, a, wall, on, the, U.S.-Mexico, bord..."
1,11685.json,FALSE,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference,"[Wisconsin, is, on, pace, to, double, the, num..."
2,11096.json,FALSE,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.,"[Says, John, McCain, has, done, nothing, to, h..."
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show,"[Suzanne, Bonamici, supports, a, plan, that, w..."
4,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video,"[When, asked, by, a, reporter, whether, hes, a..."
5,5962.json,TRUE,Over the past five years the federal governmen...,"federal-budget,pensions,retirement",brendan-doherty,NaN,Rhode Island,republican,1,2,1,1,0,a campaign website,"[Over, the, past, five, years, the, federal, g..."
6,7070.json,TRUE,Says that Tennessee law requires that schools ...,"county-budget,county-government,education,taxes",stand-children-tennessee,Child and education advocacy organization.,Tennessee,none,0,0,0,0,0,in a post on Facebook.,"[Says, that, Tennessee, law, requires, that, s..."
7,1046.json,barely-true,"Says Vice President Joe Biden ""admits that the...","economy,stimulus",john-boehner,Speaker of the House of Representatives,Ohio,republican,13,22,11,4,2,a press release.,"[Says, Vice, President, Joe, Biden, ``, admits..."
8,12849.json,TRUE,Donald Trump is against marriage equality. He ...,"gays-and-lesbians,marriage",sean-patrick-maloney,Congressman for NY-18,New York,democrat,0,0,0,0,0,a speech at the Democratic National Convention,"[Donald, Trump, is, against, marriage, equalit..."
9,13270.json,barely-true,We know that more than half of Hillary Clinton...,foreign-policy,mike-pence,Governor,Indiana,republican,8,10,12,5,0,"comments on ""Meet the Press""","[We, know, that, more, than, half, of, Hillary..."


In [51]:
df['tokenized_statement']

0    [Building, a, wall, on, the, U.S.-Mexico, bord...
1    [Wisconsin, is, on, pace, to, double, the, num...
2    [Says, John, McCain, has, done, nothing, to, h...
3    [Suzanne, Bonamici, supports, a, plan, that, w...
4    [When, asked, by, a, reporter, whether, hes, a...
5    [Over, the, past, five, years, the, federal, g...
6    [Says, that, Tennessee, law, requires, that, s...
7    [Says, Vice, President, Joe, Biden, ``, admits...
8    [Donald, Trump, is, against, marriage, equalit...
9    [We, know, that, more, than, half, of, Hillary...
Name: tokenized_statement, dtype: object

In [0]:
# Stopwords for the tokenized words
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [61]:
lemmatized = [clean(doc).split() for doc in df['Statement']]
lemmatized

[['building', 'wall', 'usmexico', 'border', 'take', 'literally', 'year'],
 ['wisconsin', 'pace', 'double', 'number', 'layoff', 'year'],
 ['say', 'john', 'mccain', 'done', 'nothing', 'help', 'vet'],
 ['suzanne',
  'bonamici',
  'support',
  'plan',
  'cut',
  'choice',
  'medicare',
  'advantage',
  'senior'],
 ['asked',
  'reporter',
  'whether',
  'he',
  'center',
  'criminal',
  'scheme',
  'violate',
  'campaign',
  'law',
  'gov',
  'scott',
  'walker',
  'nodded',
  'yes'],
 ['past',
  'five',
  'year',
  'federal',
  'government',
  'paid',
  '601',
  'million',
  'retirement',
  'disability',
  'benefit',
  'deceased',
  'former',
  'federal',
  'employee'],
 ['say',
  'tennessee',
  'law',
  'requires',
  'school',
  'receive',
  'half',
  'proceeds',
  '31',
  'million',
  'per',
  'year',
  'halfcent',
  'increase',
  'shelby',
  'county',
  'sale',
  'tax'],
 ['say',
  'vice',
  'president',
  'joe',
  'biden',
  'admits',
  'american',
  'people',
  'scammed',
  'economic'

In [65]:
# Create dictionary and term matrix
dictionary = corpora.Dictionary(lemmatized)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in lemmatized]
doc_term_matrix

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)],
 [(6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)],
 [(12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)],
 [(19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1)],
 [(28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1)],
 [(6, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 2),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1)],
 [(6, 1),
  (17, 1),
  (34, 1),
  (52, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1)],
 [(17, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1)],
 [(81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1)],
 [(58